# Week 6 - Classification models  

## Part 3: Travel mode choice - Probit regression

In this part we will revisit our real world problem of travel model choice. 

The first part is very similar to previous notebook for part 2: loading data, preprocessing, train/test split, etc. However, in this part, we will consider a Probit regression model. For the sake of simplicty, lets assume that we are just interested in distinguishing between car vs non-car (binary classification problem).

Lets just start running the parts corresponding to imports, data loading, preprocessing, train/test split, etc.

Import required libraries:

In [10]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn import linear_model
import pystan
import pystan_utils

import torch

import pyro
import pyro.distributions as dist
import pyro.poutine as poutine
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal
from pyro.infer import config_enumerate, MCMC, NUTS, HMC, SVI, Trace_ELBO
from pyro.optim import Adam

# fix random generator seed (for reproducibility of results)
np.random.seed(42)

# matplotlib style options
plt.style.use('ggplot')
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 10)

Load data:

In [2]:
# load csv
df = pd.read_csv("modechoice_data.csv")
df.head()

,Unnamed: 0,individual,hinc,psize,ttme_air,invc_air,invt_air,gc_air,ttme_train,invc_train,invt_train,gc_train,ttme_bus,invc_bus,invt_bus,gc_bus,invc_car,invt_car,gc_car,mode_chosen
0,0,70.0,30.0,4.0,10.0,61.0,80.0,73.0,44.0,24.0,350.0,77.0,53.0,19.0,395.0,79.0,4.0,314.0,52.0,1.0
1,1,8.0,15.0,4.0,64.0,48.0,154.0,71.0,55.0,25.0,360.0,80.0,53.0,14.0,462.0,84.0,4.0,351.0,57.0,2.0
2,2,62.0,35.0,2.0,64.0,58.0,74.0,69.0,30.0,21.0,295.0,66.0,53.0,24.0,389.0,83.0,7.0,315.0,55.0,2.0
3,3,61.0,40.0,3.0,45.0,75.0,75.0,96.0,44.0,33.0,418.0,96.0,53.0,28.0,463.0,98.0,5.0,291.0,49.0,1.0
4,4,27.0,70.0,1.0,20.0,106.0,190.0,127.0,34.0,72.0,659.0,143.0,35.0,33.0,653.0,104.0,44.0,592.0,108.0,1.0


Preprocess data:

In [3]:
# separate between features/inputs (X) and target/output variables (y)
mat = df.values
X = mat[:,2:-1]
print(X.shape)
y = mat[:,-1].astype("int")
print(y.shape)
ind = mat[:,1].astype("int")
print(ind.shape)

(394, 17)
(394,)
(394,)


### This part is important!

This is where we turn our previous 4-class problem into a binary classification problem: car vs non-car

In [4]:
# transform to binary problem: car vs non-car
y = (y == 4).astype("int")

In [5]:
# standardize input features
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X = (X - X_mean) / X_std

Train/test split:

In [6]:
train_perc = 0.66 # percentage of training data
split_point = int(train_perc*len(y))
perm = np.random.permutation(len(y))
ix_train = perm[:split_point]
ix_test = perm[split_point:]
X_train = X[ix_train,:]
X_test = X[ix_test,:]
y_train = y[ix_train]
y_test = y[ix_test]
print("num train: %d" % len(y_train))
print("num test: %d" % len(y_test))

num train: 260
num test: 134


Again, for the purpose of comparison, we run the logistic regression method from sklearn. But note that although sklearn has an implementation of logistic regression, it is not a Bayesian approach, nor does it support probit regression or some other variant that you may think is more appropriate for your particular problem. On the other hand, STAN offers us complete flexibility!

In [7]:
# create and fit logistic regression model
logreg = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
logreg.fit(X_train, y_train)

# make predictions for test set
y_hat = logreg.predict(X_test)
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0
 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 1 0 1
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1]
true values: [1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1]
Accuracy: 0.7164179104477612


Ok, time to implement binary logistic regression in STAN!

Your turn now :-)

Note: don't forget to include an explicit intercept parameter $\alpha$ in the model!

In [9]:
# define Stan model
model_definition = """
data {
  int<lower=0> N;    // shape of data
  int<lower=0> D;    // dimensions of x
  matrix[N,D]  X;    // feature matrix
  int          y[N]; // response vector
}
parameters {
  real      alpha; // bias 
  vector[D] beta;  // parameters
}
model{
  beta   ~ normal(0, 5);
  alpha  ~ normal(0., 5);
  
  y ~ bernoulli_logit(alpha + X*beta);  
}
"""

# compile model
sm = pystan.StanModel(model_code=model_definition)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_54158a9c63a2718c46123f79ff152fa0 NOW.


In [8]:
def model(X, obs=None):
    n_beta = X.shape[1]
    alpha = pyro.sample("alpha", dist.Normal(0., 5.))     # Prior for the bias
    beta  = pyro.sample("beta", dist.Normal(0., 5.).expand([n_beta]).to_event(1)) # Priors for the regression coefficents
    
    mu = alpha + X.matmul(beta)
    with pyro.plate("data", X.shape[0]):
        y = pyro.sample("y", dist.Bernoulli(logits=mu), obs=obs) # If you use logits you don't need to do sigmoid
        
    return y

In [11]:
n_cat = 4
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()

In [16]:
#hmc_kernel = HMC(model)
nuts_kernel = NUTS(model)
mcmc = MCMC(nuts_kernel, num_samples=1000, warmup_steps=100, num_chains=1)
mcmc.run(X_train, y_train) # Pyro accepts categories starting from 0

Sample: 100%|██████████| 1100/1100 [06:27,  2.84it/s, step size=2.35e-02, acc. prob=0.958]


In [17]:
mcmc.summary()


                mean       std    median      5.0%     95.0%     n_eff     r_hat
     alpha     -0.66      0.16     -0.66     -0.95     -0.42   1109.65      1.00
   beta[0]      0.59      0.19      0.59      0.29      0.89   1048.14      1.00
   beta[1]      0.24      0.28      0.23     -0.22      0.70   1026.03      1.00
   beta[2]      0.63      0.17      0.62      0.37      0.91   1099.55      1.00
   beta[3]      4.15      2.32      4.16      0.66      8.03    441.00      1.00
   beta[4]      0.91      0.57      0.91     -0.02      1.77    517.49      1.00
   beta[5]     -4.53      2.52     -4.55     -8.32     -0.44    436.13      1.00
   beta[6]      0.19      0.18      0.19     -0.12      0.46    906.76      1.00
   beta[7]     -0.15      1.37     -0.18     -2.23      2.34    499.85      1.01
   beta[8]      3.52      1.81      3.46      0.58      6.20    486.14      1.00
   beta[9]     -0.83      2.85     -0.70     -5.27      4.09    508.96      1.00
  beta[10]      0.68      0

Prepare input data for STAN, compile STAN program and run inference (MCMC):

In [10]:
# prepare data for Stan model
N, D = X_train.shape
print("N=%d, D=%d" % (N,D))
data = {'N': N, 'D': D, 'X': X_train, 'y': y_train}

N=260, D=17, C=1


In [11]:
%%time
fit = sm.sampling(data=data, iter=1000, chains=4, algorithm="NUTS", seed=42, verbose=True)

CPU times: user 56.6 ms, sys: 34.1 ms, total: 90.7 ms
Wall time: 16.3 s


In [12]:
print(fit)

Inference for Stan model: anon_model_54158a9c63a2718c46123f79ff152fa0.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha     -0.67  3.5e-3   0.16  -0.97  -0.77  -0.67  -0.56  -0.36   2073    1.0
beta[1]    0.59  4.1e-3    0.2   0.21   0.45   0.59   0.72    1.0   2427    1.0
beta[2]    0.25  5.5e-3   0.29  -0.32   0.05   0.24   0.44   0.81   2762    1.0
beta[3]    0.62  3.6e-3   0.17   0.29    0.5   0.62   0.73   0.96   2254    1.0
beta[4]    3.94    0.07   2.26  -0.41   2.35   3.91   5.47   8.27   1116    1.0
beta[5]    0.86    0.02   0.55  -0.23   0.48   0.86   1.22   1.97   1196    1.0
beta[6]   -4.33    0.07   2.44   -9.1  -5.95  -4.31  -2.63   0.36   1121    1.0
beta[7]    0.18  3.5e-3   0.17  -0.15   0.07   0.18   0.29   0.53   2381    1.0
beta[8]   -0.29    0.04   1.46  -3.33  -1.27  -0.29   0.72   2.61   1073    1.0
beta[9]    3.

Extract samples from posterior, make predictions and compute accuracy (make sure that you understand all the code!):

In [13]:
samples = fit.extract(permuted=True)  # return a dictionary of arrays

In [14]:
# make predictions for test set
mu = np.mean(samples["alpha"].T + np.dot(X_test, samples["beta"].T), axis=1)
y_hat = (mu > 0).astype("int")
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1
 1 1 0 0 0 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 0 0 0 1 1 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
true values: [1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1]
Accuracy: 0.7313432835820896


Nice, it seems that we are already doing better than sklearn!

Ok, now lets try a **probit regression model in STAN**.

Can you implement it?

Note: the function that you need to use for the probit is called *Phi_approx* in STAN

In [21]:
# define Stan model
model_definition = """
data {
  int<lower=0> N;    // shape of data
  int<lower=0> D;    // dimensions of x
  int<lower=0> C;    // number of possible choices
  matrix[N,D]  X;    // feature matrix
  int          y[N]; // response vector
}
parameters {
  real      alpha; // bias 
  vector[D] beta;  // parameters
}
model{
  vector[N] mu;
  
  beta   ~ normal(0, 5);
  alpha  ~ normal(0, 5);
  
  mu = alpha + X*beta; 
  for (n in 1:N) {
    mu[n] <- Phi_approx(mu[n]);
  }
  
  y ~ bernoulli(mu);  
}
"""

# compile model
sm = pystan.StanModel(model_code=model_definition)

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_f920a7b1d80ae11b599f4fcc0f87fdb8 NOW.


Prepare input data for STAN, compile STAN program and run inference (MCMC):

In [22]:
# prepare data for Stan model
N, D = X_train.shape
print("N=%d, D=%d" % (N,D))
data = {'N': N, 'D': D, 'X': X_train, 'y': y_train}

N=260, D=17, C=1


In [23]:
%%time
fit = sm.sampling(data=data, iter=1000, chains=4, algorithm="NUTS", seed=42, verbose=True)

CPU times: user 70.2 ms, sys: 28.6 ms, total: 98.7 ms
Wall time: 22.9 s


In [24]:
print(fit)

Inference for Stan model: anon_model_f920a7b1d80ae11b599f4fcc0f87fdb8.
4 chains, each with iter=1000; warmup=500; thin=1; 
post-warmup draws per chain=500, total post-warmup draws=2000.

           mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
alpha     -0.37  1.9e-3    0.1  -0.56  -0.43  -0.37   -0.3  -0.19   2558    1.0
beta[1]    0.37  2.5e-3   0.12   0.14   0.28   0.37   0.45    0.6   2435    1.0
beta[2]    0.17  3.4e-3   0.17  -0.18   0.05   0.17   0.29    0.5   2712    1.0
beta[3]    0.39  2.0e-3   0.11   0.18   0.31   0.39   0.46    0.6   2859    1.0
beta[4]    3.39    0.05   1.71   0.14   2.21   3.34   4.56   6.76   1132    1.0
beta[5]    0.74    0.01    0.4-9.1e-3   0.46   0.74    1.0   1.54   1181    1.0
beta[6]   -3.66    0.05   1.86  -7.33  -4.94  -3.62  -2.34  -0.14   1141    1.0
beta[7]    0.11  2.1e-3   0.11   -0.1   0.05   0.11   0.18   0.33   2647    1.0
beta[8]   -0.08    0.05   1.29  -2.63  -0.95  -0.09   0.82   2.43    719    1.0
beta[9]    2.

Extract samples from posterior, make predictions and compute accuracy (make sure that you understand all the code!):

In [25]:
samples = fit.extract(permuted=True)  # return a dictionary of arrays

In [26]:
# make predictions for test set
mu = np.mean(samples["alpha"].T + np.dot(X_test, samples["beta"].T), axis=1)
y_hat = (mu > 0).astype("int")
print("predictions:", y_hat)
print("true values:", y_test)

# evaluate prediction accuracy
print("Accuracy:", 1.0*np.sum(y_hat == y_test) / len(y_test))

predictions: [0 0 0 1 1 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 1 0 1 1 1 0 0 1 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 0 0 0 0 0 1 1 1
 1 1 0 0 1 0 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 1 1 0 0 1 1 0 0 0 1
 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1]
true values: [1 0 0 1 1 0 0 0 1 1 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 1 0 0 0 0 1 1 1 1 0 0 0
 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 0 1 1 1 1 0 0 0 0 0 0 1 0 1
 1 1 0 0 1 0 1 1 0 1 0 0 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 0 0 0 0 0 1 1 1 0 1
 1 1 0 1 0 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 1]
Accuracy: 0.7164179104477612


How are your results in comparison to the version with the logistic sigmoid?

In some cases, using a probit function instead of the logistic sigmoid can make a significant difference. In other cases, it doesn't... You have to consider what makes more sense to the specific problem that you are trying to solve. Or, we can just try different approaches! That is just fine... STAN makes it very easy to try all these different variants.